Marine Phosphate Cycling and primary productivity
=================================================



## Notes:



-   This `Jupyter Notebook` contains text cells, and code cells.
-   You can edit the text cells using markdown syntax. Just double click this cell to try it out. When finished, press shift+enter.
-   You can execute the python code in each text cell by placing the cursor into a code-cell, and press shift+enter.
-   If you have more then one code cell, you need to execute them in sequential
    order since each cell depends on the output of the previous cell.



## Model structure



In the following, we use a two box ocean model. The surface box represents the `photic zone`  of the ocean, and the deep-box represents the `deep ocean` that is below the photic zone.  The mode approximates the `thermohaline circulation` by two fluxes, $F_u$ which describes the upwelling of deep water, and $F_d$ which describes the downwelling of surface water.

Marine primary productivity (i.e., photosynthesis) is typically limited by the availability of dissolved phosphate. In order to keep the model simple, we assume a linear relationship between the Phosphate concentration and the amount of `organic matter` that is produced by photosynthesis. Since we are only interested in Phosphate, there are no terms for organic matter (OM) production. However, since OM will eventually sink into the deep ocean, the organic bound Phosphate will also sink into the deep ocean. I will call this settling flux $F_{POP}$ which stands for "Particulate Organic Phosphate".  

This flux will depend on the concentration of phosphate, and on the speed how fast organisms can metabolize phosphate (aka phosphate uptake efficiency). There a different ways to describe the uptake efficiency. We could, e.g., state that 70% of the phosphate that is supplied through upwelling ($F_u$) is used for organic matter production. Another way is to state that all phosphate reaching the surface ocean is used within 1 year. In the following we will describe the uptake efficiency as a function of the phosphate residency time in the surface ocean. So $F_{POP}$ is being proportional to the PO<sub>4</sub> concentration and the uptake efficiency $V_S/\tau$:

\begin{equation}\label{}
F_{POP} = [PO_4]_S \times \frac{V_S}{\tau}
\end{equation}

where $\tau$ is the residence of phosphate in the surface ocean and $V_S$ is the volume of the surface box.

We can now describe the equations that govern [PO<sub>4</sub>] in the surface and deep ocean

![img](./images/mpc.png)nf

Similar to the previous  model, our fluxes are not static (with the exception of the weathering flux). $F_u$ and $F_b$ depend on the velocity of thermohaline circulation, and the respective phosphate concentration in each box:

\begin{equation}\label{}
F_d = thc \times [PO_4]S_{}
\end{equation}

where `thc` denotes the velocity of  the thermohaline circulation, and $[PO_4]_D$ the concentration in the deep ocean.  A similar equation can be written for $F_d$. 



#### The burial flux



Most of the sinking organic matter will be `remineralized` in the deep
ocean. For our model, we sill assume that the remineralization efficiency is
about 99% percent. I.e., only 1% percent of the marine export productivity is
buried in the sediments. We can thus write:

\begin{equation}\label{}
F_b = F_{POP} \times (1 -\epsilon)
\end{equation}

where $\epsilon$ denotes the burial efficiency (i.e., 99% = 0.99). 



### Model Equations



For the PO4 concentration in the surface ocean, we can write:

\begin{equation}\label{}
\frac{[PO_{4}]_{S}}{dt} = F_w + [PO_{4}]_{D} \times thc - [PO_{4}]_{S} \times thc - [PO_{4}]_{S} \times \frac{V_S}{\tau}
\end{equation}

and for the deep ocean:

\begin{equation}\label{}
\frac{[PO_4]_S}{dt} =  [PO_4]_S \times \frac{V_S}{\tau} \times \epsilon + [PO_4]_{S} \times thc - [PO_4]_{D} \times thc
\end{equation}

Note that both equations depend on each other, i.e., we need to know the result from one equation to calculate the result of the other equation (i.e., coupled differential equations).



## Model boundary conditions



As before, use your notes to find the missing boundary condition data below, and edit the code accordingly. Then execute the code cell, and verify the values before you continue.

Note if your values are way off, the model will either not run, or run forever. So check that F<sub>r</sub> is between 1E3 and 2E3 mol/s, and that your surface ocean volume is between 20E15 and 40E15 m<sup>2</sup>, and the deep ocean volume is between 1E18 and 2E18 m<sup>3</sup>. THC must be between 20E6 and 40E6. If in doubt check with instructor.



In [1]:
"""
A simple P-cycle model, based on chapter 8 of Modeling Methods for the marine
sciences.

This model uses a 2 box ocean, and the P-export flux depends on the biological
productivity in the surface ocean
"""
import po4

F_r =   # River (weathering) flux of PO4 mol/s
V = [, ]  # Volume of surface and deep box m^3 [1, 3]
THC =   # m^3/s
tau =   # P residency time in surface ocean in years
r =   # remineralization efficiency
C_0 = [0, 0]  # initial P in surface and deep box
time = 4e6  # run time in years
plot_dpi = 300  #
figure_name = "po42.png"

# ---------- no user serviceable parts below ---------- #
print(f"Vs = {V[0]:.2e} m^3, C_0_s = {C_0[0]:.2f} mol/m^3")
print(f"Vd = {V[1]:.2e} m^3, C_0_d = {C_0[1]:.2f} mol/m^3")
print(f"tau = {tau:2e} yrs, r = {r:.2f}, run time = {time:.2e} yrs")

fig = po4.po42(time, C_0, V, F_r, plot_dpi, figure_name, tau, r, THC)

## Execute the model



You can then execute the code by clicking on the double sideways triangle icon (similar to fast forward on a TV remote) on the top of the page.  You should see a graph describing how the marine PO<sub>4</sub> concentration changes with time. Now lets do some sanity checks. Your total phosphate mass should be about 2.85e+16 mol, and your residence time should be around 601 kyrs. If not, there is a mistake in the boundary conditions.

